# Preprocessing

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)

In [2]:
merged_df = pd.read_csv("../00_data/1_interim/merged_data.csv", index_col=["Code", "Year"])

### Handle High Missing Values #1: by Code (Country ID)

In [3]:
# Missing Values by Country ID (Code)
all_missing_values = merged_df.isnull().groupby(level=0).sum()
# 9 main potential features: 
values_count_per_country = merged_df.groupby(level=0).size().iloc[0] * 9

all_missing_values["total_missing"] = all_missing_values.sum(axis=1)
all_missing_values["total_missing_%"] = round( ( all_missing_values["total_missing"] / values_count_per_country ) * 100, 2)
#plot top 12 countries with most missing values
top_missing_countries = all_missing_values.sort_values(ascending=False, by="total_missing")
top_12_missing = top_missing_countries["total_missing"].head(12)
print(f"Top 12 Countries with most missing values:\n{top_12_missing}")
#Threshold 50%: all countries with missing values over 50% get excluded from Dataframe
# 9 x 6 = 54 data point for each country --> 27 NaNs as upper limit: 
# AIA       49 (Anguilla)
# MSR       48 (Montserrat)
# OWID_KOS  48 (Kosovo)
# COK       42 (Cook Islands)
# NIU       42 (Niue)
# TCA       39 (Turks and Caicos Islands)
# VGB       38 (British Virgin Islands)

Top 12 Countries with most missing values:
Code
AIA         49
MSR         48
OWID_KOS    48
COK         42
NIU         42
TCA         39
VGB         38
PRK         26
SSD         26
MCO         25
GNQ         22
VEN         21
Name: total_missing, dtype: int64


In [4]:
# Exclude high missing values countries (threshold >= 50%) & save as new df
# new filtered version has now 967 missing values (306 less than before)
exclude_countries = top_missing_countries[top_missing_countries["total_missing_%"] >= 50]
filtered_df = merged_df[~merged_df.index.get_level_values(0).isin(exclude_countries.index.tolist())].copy()
filtered_df.to_csv('../00_data/1_interim/filtered_data_01.csv', index=True)
filtered_df.isna().sum().sum()

np.int64(967)

In [5]:
filtered_df.isna().sum()

Entity                                        0
child_mortality_igme                          0
annual_healthcare_expenditure_per_capita     16
gdp_per_capita_worldbank                     42
nurses_and_midwives_per_1000_people         300
physicians_per_1000_people                  405
prevalence_of_undernourishment              156
share_of_population_urban                     0
share_without_improved_water                 16
vaccination_coverage_who_unicef               6
years_of_schooling                           26
dtype: int64

### Testing: Handle Missing Values #2: Missing Indikators

In [6]:
missing_indicators_df = filtered_df.copy()
# add _missing indikator: 0 if not missing, 1 if missing
for column in missing_indicators_df.columns:
    if missing_indicators_df[column].isna().sum() > 0:
        missing_indicators_df[column+"_missing"] = missing_indicators_df[column].isna().astype(int)

# Data not missing at random > include missing indicator values to trainingset in pipeline later

In [7]:
missing_indicators_df

Entity  child_mortality_igme  \
Code Year                                      
AFG  2013  Afghanistan             78.022990   
     2014  Afghanistan             75.123625   
     2015  Afghanistan             72.438707   
     2016  Afghanistan             69.970040   
     2017  Afghanistan             67.637370   
...                ...                   ...   
ZWE  2014     Zimbabwe             61.771426   
     2015     Zimbabwe             59.763910   
     2016     Zimbabwe             56.912365   
     2017     Zimbabwe             54.953800   
     2018     Zimbabwe             52.289700   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
AFG  2013                                 177.62009                 3046.5798   
     2014                                 201.79086                 3017.9426   
     2015                                 215.22618                 2967.6921   
     2016                                 238.74481                 2958.7854   
     2017                                 264.20660                 2952.9990   
...                                             ...                       ...   
ZWE  2014                                 203.30057                 3352.3813   
     2015                                 191.68187                 3366.6338   
     2016                                 194.29993                 3345.3150   
     2017                                 143.32605                 3453.5059   
     2018                                 114.42487                 3572.9814   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
AFG  2013                                0.255                       0.291   
     2014                                0.151                       0.304   
     2015                                0.133                       0.291   
     2016                                0.151                       0.284   
     2017                                0.179                       0.245   
...                                        ...                         ...   
ZWE  2014                                1.507                       0.122   
     2015                                1.349                       0.177   
     2016                                1.359                       0.174   
     2017                                2.507                       0.179   
     2018                                1.852                       0.201   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                                              
AFG  2013                            19.4                     24.373   
     2014                            19.3                     24.587   
     2015                            20.0                     24.803   
     2016                            20.5                     25.020   
     2017                            21.4                     25.250   
...                                   ...                        ...   
ZWE  2014                            30.4                     32.504   
     2015                            33.0                     32.385   
     2016                            35.5                     32.296   
     2017                            36.7                     32.237   
     2018                            38.7                     32.209   

           share_without_improved_water  vaccination_coverage_who_unicef  \
Code Year                                                                  
AFG  2013                     38.863914                             64.0   
     2014                     36.337383                             62.0   
     2015                     33.812515                             64.0   
     